In [34]:
import pandas as pd
import numpy as np
import os
import sys
import nltk
import contractions
import string 
from pathlib import Path
import re

In [145]:
pd.options.display.max_colwidth = None


In [3]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/shaul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
PATH_ROOT = Path(os.getcwd()).parents[1].resolve()
PATH_MAIN = PATH_ROOT / 'data' / 'full_DS' / 'full_metrics.csv'
PATH_STS = PATH_ROOT / 'data' / 'eval_data' / 'sts.csv'
PATH_QQP = PATH_ROOT / 'data' / 'eval_data' / 'qqp.csv'

## First Dataset

In [104]:
main_df = pd.read_csv(PATH_QQP, index_col=0)
main_df.columns

Index(['pair_id', 'text_1', 'text_2', 'label', 'dataset', 'bleu', 'bleu1',
       'glove_cosine', 'fasttext_cosine', 'chrfScore', 'POS Dist score', 'WMD',
       '1-gram_overlap', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l', 'BertScore',
       'L2_score'],
      dtype='object')

In [105]:
group_df = main_df.groupby('pair_id')[['text_1','text_2']].last()
main_texts = pd.Series(group_df['text_1'].tolist() + group_df['text_2'].tolist())

In [92]:
# Are there null values
print(f"Are there Null values: {main_texts.isnull().any()}")

#Are there any empty sentences once we remove punctuation
pattern = f"[{string.punctuation}]"
filtered_texts = main_texts.apply(lambda x: re.sub(pattern," ",x))
filtered_texts = filtered_texts.apply(lambda x: re.sub(" +"," ",x))
for iloc in np.argwhere((filtered_texts.apply(len) < 5).values).flatten():
    print(main_texts.iloc[iloc])
    print(filtered_texts.iloc[iloc])

Are there Null values: False


In [93]:
uncontracted_texts = main_texts.apply(lambda x: contractions.fix(x,slang=False))
for i, example in enumerate(np.where((uncontracted_texts.apply(len) != main_texts.apply(len)).values)[0]):
    print(main_texts.iloc[example])
    print(uncontracted_texts.iloc[example])
    print("\n")
    if i > 20:
        break

In [149]:
# Does it have latex math equations
for iloc in np.where(main_texts.apply(lambda x: re.findall("\[math\]","".join(x)))):
    print(main_texts.iloc[iloc])

362                                     How do I find the zeros of the polynomial function [math]f(x)=\dfrac{1}{2}x^{3}-3x[/math]?
764                                                         What is [math]x[/math] if [math]x+\left(\dfrac{1}{x}\right) =0[/math]?
1275                                                   How do I integrate [math]v^3(a^2-v^2)^{\frac{1}{2}}[/math] by substitution?
1658                                                                                  What is the formula of [math](a+b)^3[/math]?
2712                                                      What is [math]\sqrt {2+\sqrt {2+\sqrt {2+\sqrt {2+...\infty}}}}[/math] ?
                                                                    ...                                                           
802503                                   What is limit of [math]\dfrac{1^4+2^4+3^4....n^4}{n^4}[/math]? where n tends to infinity?
803217                                                                             

In [150]:
# Explore cases where there is a repitition of punctuation (3+)
for iloc in np.where(main_texts.apply(lambda x: re.findall(pattern + "{3,}","".join(x)))):
    print(main_texts.iloc[iloc])

5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
37                                                                                                                                                                                                                                                                    

In [152]:
# Explore cases where we have characters outside of the regular language?
for iloc in np.where(main_texts.apply(lambda x: re.findall(f"[^{string.ascii_letters}{string.digits}{string.punctuation} ]","".join(x)))):
    # print(re.sub(f"[^{string.ascii_letters}{string.digits}{string.punctuation} ]"," ",str(main_texts.iloc[iloc])))
    print(main_texts.iloc[iloc])

8                                                                                                                                When do you use シ instead of し?
15                                                                 What would a Trump presidency mean for current international master’s students on an F1 visa?
161       I got job offer @ Chelmsford-Essex, London with £3764 PM pay-after tax deduction. Pls advice tentative monthly expenses for couple? & saving possible?
283                                  When travelling to a new region is it better to immerse yourself in 1–2 cities or to see as many cities as you can cram in?
362                                                                   How do I find the zeros of the polynomial function [math]f(x)=\dfrac{1}{2}x^{3}-3x[/math]?
                                                                                   ...                                                                          
807697                            

In [203]:
# Explore for Non-Ascii characters
import unicodedata
for iloc in np.where(main_texts.apply(lambda x: re.findall("[^\x00-\x7F]",x)))[0].flatten():
    print(unicodedata.normalize('NFKD',str(main_texts.iloc[iloc])).encode('ascii','ignore').decode())
    print(main_texts.iloc[iloc])
    print("\n")

actually asked that question, although I might have answered it?
Why am I being told by Quora "Your question X may need editing …" when I never actually asked that question, although I might have answered it?


I sent a message to a friend in Facebook but immediately deleted it before it got sent. My connection is poor so its slower. Will it be delivered?
I sent a message to a friend in Facebook but immediately deleted it before it got sent. My connection is poor so it’s slower. Will it be delivered?


What is your 2017 New Years resolution?
What is your 2017 New Year’s resolution?


What strikes first time visitors as special or unusual when they arrive in Chillan, Chile?
What strikes first time visitors as special or unusual when they arrive in Chillán, Chile?


What does David Bowies gesture on the cover of the album "Heroes" mean?
What does David Bowie’s gesture on the cover of the album "Heroes" mean?


1. How many binary strings of length 9 are there such that there are three 1s 

In [204]:
# Explore for Non-Ascii characters
all_iloc = np.where(main_texts.apply(lambda x: re.findall("[^\x00-\x7F]","".join(x))))[0].flatten()

sum(main_texts.iloc[all_iloc].apply(lambda x: unicodedata.normalize('NFKD',str(x)).encode('ascii','ignore').decode()).apply(len) < main_texts.iloc[all_iloc].apply(len))

7220